# Home Page

In [ ]:
import dash
from dash import dcc, html
import base64
from dash.dependencies import Input, Output

# Dash app initialization
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True

# Define image paths
background_image_path = '/content/drive/MyDrive/rs_project/assets/i9.png'
small_image_path = '/content/drive/MyDrive/rs_project/assets/logo.png'
additional_image_path_1 = '/content/drive/MyDrive/rs_project/assets/user.png'
additional_image_path_2 = '/content/drive/MyDrive/rs_project/assets/movie.png'

# Encode images to base64
with open(background_image_path, 'rb') as bg_file:
    encoded_bg_image = base64.b64encode(bg_file.read()).decode('ascii')

with open(small_image_path, 'rb') as sm_file:
    encoded_small_image = base64.b64encode(sm_file.read()).decode('ascii')

with open(additional_image_path_1, 'rb') as add_file_1:
    encoded_additional_image_1 = base64.b64encode(add_file_1.read()).decode('ascii')

with open(additional_image_path_2, 'rb') as add_file_2:
    encoded_additional_image_2 = base64.b64encode(add_file_2.read()).decode('ascii')

# Layout for the home page
home_layout = html.Div(
    style={'backgroundImage': f'url(data:image/jpg;base64,{encoded_bg_image})',
           'backgroundSize': 'cover',
           'backgroundPosition': 'center',
           'height': '100vh',
           'position': 'relative'},
    children=[
        html.Img(src=f'data:image/png;base64,{encoded_small_image}',
                 style={'position': 'absolute',
                        'top': '30%',
                        'right': '90px',
                        'transform': 'translateY(-50%)'}),
        dcc.Link(
            html.Img(src=f'data:image/png;base64,{encoded_additional_image_1}',
                     style={'position': 'absolute',
                            'bottom': '20px',
                            'left': '60px',
                            'width': '500px'}),  # Adjust width as needed
            href='/user'
        ),
        dcc.Link(
            html.Img(src=f'data:image/png;base64,{encoded_additional_image_2}',
                     style={'position': 'absolute',
                            'bottom': '20px',
                            'right': '60px',
                            'width': '500px'}),  # Adjust width as needed
            href='/movie'
        )
    ]
)

# Layout for the user page
user_layout = html.Div([
    html.H1("User Page"),
    dcc.Link('Go back to home', href='/')
])

# Layout for the movie page
movie_layout = html.Div([
    html.H1("Movie Page"),
    dcc.Link('Go back to home', href='/')
])

# Define the app layout with a router
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Update page content based on URL
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/user':
        return user_layout
    elif pathname == '/movie':
        return movie_layout
    else:
        return home_layout

if __name__ == '__main__':
    app.run_server(debug=True,jupyter_mode= 'external')


# User Page

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import requests
import tensorflow as tf
import pickle
from libreco.algorithms import NCF
from PIL import Image, ImageFilter
import base64
from io import BytesIO

tf.keras.backend.clear_session()
# Function to convert image to base64
def image_to_base64(image):
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    encoded_image = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return encoded_image

# Function to fetch movie details
def fetch_movie_details(imdbId, api_key):
    try:
        imdbId = str(imdbId).zfill(7)  # Ensure the IMDb ID is 7 digits long
        url = f"http://www.omdbapi.com/?i=tt{imdbId}&apikey={api_key}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data['Response'] == 'True':
                return data
            else:
                return None
        else:
            return None
    except Exception as e:
        return None

# Your OMDb API key
api_key = '439d9f47'

# Load the data_info object
with open('/content/drive/MyDrive/rs_project/data_info.pkl', 'rb') as f:
    data_info = pickle.load(f)

# Load the NCF model
model_path = "/content/drive/MyDrive/rs_project/ncf_user_model.h5"
ncf_loaded = NCF.load(model_path, model_name="ncf_model", data_info=data_info)

# Load the datasets
ratings = pd.read_csv('/content/drive/MyDrive/rs_project/ml-latest-small/ratings.csv')
movies_df = pd.read_csv('/content/drive/MyDrive/rs_project/ml-latest-small/movies.csv')
links_df = pd.read_csv('/content/drive/MyDrive/rs_project/ml-latest-small/links.csv')

# Merge links_df with movies_df to include IMDb IDs
movies_df = pd.merge(movies_df, links_df[['movieId', 'imdbId']], on='movieId')

# Load the background image
bg_image = Image.open("/content/drive/MyDrive/rs_project/assets/background.png")
blurred_bg = bg_image.filter(ImageFilter.BLUR)

# Load the logo image
logo_image = Image.open("/content/drive/MyDrive/rs_project/assets/logo.png")
encoded_logo = image_to_base64(logo_image)



# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div(style={'background-image': f'url("data:image/png;base64,{image_to_base64(blurred_bg)}")', 'background-size': 'cover'},
    children=[
        html.Div(
            children=[
                html.Img(src=f'data:image/png;base64,{encoded_logo}', style={'position': 'absolute', 'top': '30px', 'left': '10px', 'height': '50px'}),
                html.Center(
                    children=[
                        html.H1("Find your ID", style={'color':'white','fontSize': '40px', 'fontWeight': 'bold'}),
                        dcc.Dropdown(
                            id='user-dropdown',
                            options=[{'label': str(user), 'value': user} for user in ratings['userId'].unique()],
                            placeholder="Select ID",
                            style={'width': '40%', 'marginBottom': '20px', 'color': 'black', 'zIndex': '2', 'margin': 'auto', 'fontSize': '20px'}
                        ),
                        html.H2("your history is:", style={'fontSize': '20px', 'fontStyle': 'italic', 'color':'white'}),
                        html.Div(id='user-history', style={'display': 'flex', 'overflowX': 'auto', 'whiteSpace': 'nowrap'}),
                        html.Div(id='recommendation-section', style={'display': 'none'}, children=[
                            html.H2("Movies for today", style={'fontSize': '40px', 'color': 'white', 'fontWeight': 'bold'}),
                            dcc.Dropdown(
                                id='recommendation-dropdown',
                                options=[{'label': str(i), 'value': i} for i in range(1, 21)],
                                placeholder="Select number of recommendations",
                                style={'width': '40%', 'marginBottom': '20px', 'color': 'black', 'zIndex': '2', 'margin': 'auto', 'fontSize': '20px'}
                            ),
                            html.H2("Grab a snack, we reserve a seat for you on:", style={'fontSize': '20px', 'color': 'white', 'fontStyle': 'italic'}),
                            html.Div(id='user-recommendations', style={'display': 'flex', 'overflowX': 'auto', 'whiteSpace': 'nowrap'})
                        ])
                    ]
                ),
                html.Div(id='movie-modal', style={'display': 'none'}, children=[
                    html.Div(id='modal-content', style={
                        'position': 'fixed', 'top': '50%', 'left': '50%', 'transform': 'translate(-50%, -50%)',
                        'backgroundColor': 'rgba(0,0,0,0.8)', 'padding': '20px', 'borderRadius': '10px', 'zIndex': '1000',
                        'width': '80%', 'maxWidth': '600px'
                    }, children=[
                        html.Button('Close', id='close-modal', style={'float': 'right', 'backgroundColor': 'red', 'color': 'white'}),
                        html.Div(id='modal-body')
                    ]),
                    html.Div(style={
                        'position': 'fixed', 'top': '0', 'left': '0', 'width': '100%', 'height': '100%',
                        'backgroundColor': 'rgba(0,0,0,0.5)', 'zIndex': '999'
                    })
                ])
            ]
        )
    ]
)

def get_movie_title(movie_id, movies_df):
    title = movies_df[movies_df['movieId'] == movie_id]['title'].values[0]
    return title

def get_movie_imdbId(movie_id, movies_df):
    imdbId = movies_df[movies_df['movieId'] == movie_id]['imdbId'].values[0]
    return imdbId

def user_recommendation(user_id, top_k, model, movies_df):
    # Get top k recommended movie IDs for the given user
    rec_movies = model.recommend_user(user=user_id, n_rec=top_k)

    # Create a list to store the recommendations
    recommendations = []

    for movie_id in rec_movies[user_id]:
        title = get_movie_title(movie_id, movies_df)
        imdbId = get_movie_imdbId(movie_id, movies_df)
        recommendations.append((movie_id, title, imdbId))

    return recommendations

@app.callback(
    [Output('user-history', 'children'),
     Output('recommendation-section', 'style')],
    [Input('user-dropdown', 'value')]
)
def update_user_history(selected_user):
    if selected_user is not None:
        selected_user = int(selected_user)  # Convert to int if necessary
        # Retrieve user history
        user_history = ratings[ratings['userId'] == selected_user]
        user_history_movies = user_history.merge(movies_df, on='movieId', how='inner')[['title', 'imdbId']]

        user_history_elements = []
        for _, row in user_history_movies.iterrows():
            movie_details = fetch_movie_details(row['imdbId'], api_key)
            if movie_details:
                poster = movie_details.get('Poster') if movie_details.get('Poster') and movie_details.get('Poster') != 'N/A' else ''
                user_history_elements.append(
                    html.Div(
                        children=[
                            html.Img(src=poster, style={'width': '100px', 'height': '150px', 'borderRadius': '10px', 'margin': '10px', 'cursor': 'pointer'}, id={'type': 'movie-img', 'index': row['imdbId']}),
                            html.P(row['title'], style={'textAlign': 'center', 'color': 'white'})
                        ],
                        style={'display': 'inline-block', 'margin': '10px'}
                    )
                )
            else:
                user_history_elements.append(
                    html.Div(
                        children=[
                            html.Div("Details not found", style={'width': '100px', 'height': '150px', 'borderRadius': '10px', 'margin': '10px', 'backgroundColor': 'gray', 'color': 'white', 'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'}),
                            html.P(row['title'], style={'textAlign': 'center', 'color': 'white'})
                        ],
                        style={'display': 'inline-block', 'margin': '10px'}
                    )
                )
        return user_history_elements, {'display': 'block'}
    else:
        return [], {'display': 'none'}

@app.callback(
    Output('user-recommendations', 'children'),
    [Input('user-dropdown', 'value'),
     Input('recommendation-dropdown', 'value')]
)
def update_user_recommendations(selected_user, num_recommendations):
    if selected_user is not None and num_recommendations is not None:
        try:
            selected_user = int(selected_user)  # Convert to int if necessary
            num_recommendations = int(num_recommendations)  # Convert to int if necessary

            # Retrieve user recommendations using the user_recommendation function
            user_recommendations = user_recommendation(selected_user, num_recommendations, ncf_loaded, movies_df)

            recommendation_elements = []
            for _, title, imdbId in user_recommendations:
                movie_details = fetch_movie_details(imdbId, api_key)
                if movie_details:
                    poster = movie_details.get('Poster') if movie_details.get('Poster') and movie_details.get('Poster') != 'N/A' else ''
                    recommendation_elements.append(
                        html.Div(
                            children=[
                                html.Img(src=poster, style={'width': '100px', 'height': '150px', 'borderRadius': '10px', 'margin': '10px', 'cursor': 'pointer'}, id={'type': 'movie-img', 'index': str(imdbId)}),
                                html.P(f'{title}', style={'textAlign': 'center', 'color': 'white'})
                            ],
                            style={'display': 'inline-block', 'margin': '10px'}
                        )
                    )
                else:
                    recommendation_elements.append(
                        html.Div(
                            children=[
                                html.Div("Details not found", style={'width': '100px', 'height': '150px', 'borderRadius': '10px', 'margin': '10px', 'backgroundColor': 'gray', 'color': 'white', 'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'}),
                                html.P(f'{title}', style={'textAlign': 'center', 'color': 'white'})
                            ],
                            style={'display': 'inline-block', 'margin': '10px'}
                        )
                    )

            return recommendation_elements
        except Exception as e:
            return [html.Div(f"An error occurred: {str(e)}", style={'color': 'white'})]
    else:
        return []

@app.callback(
    [Output('movie-modal', 'style'),
     Output('modal-body', 'children')],
    [Input({'type': 'movie-img', 'index': dash.dependencies.ALL}, 'n_clicks'),
     Input('close-modal', 'n_clicks')],
    [State({'type': 'movie-img', 'index': dash.dependencies.ALL}, 'id')]
)
def display_movie_modal(n_clicks, close_click, ids):
    ctx = dash.callback_context

    if not ctx.triggered:
        return {'display': 'none'}, []

    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # If the modal close button is clicked, close the modal
    if 'close-modal' in triggered_id:
        return {'display': 'none'}, []

    # Check if the triggered input is a movie image
    if 'movie-img' in triggered_id:
        movie_imdb_id = eval(triggered_id)['index']
        movie_details = fetch_movie_details(movie_imdb_id, api_key)

        if movie_details is not None:
            poster = movie_details.get('Poster') if movie_details.get('Poster') and movie_details.get('Poster') != 'N/A' else ''
            modal_content = [
                html.Div(style={
                    'background-image': f'url("{poster}")',
                    'background-size': 'cover',
                    'background-repeat': 'no-repeat',
                    'width': '100%',
                    'height': '300px',
                    'borderRadius': '10px'
                }),
                html.H2(movie_details.get('Title'), style={'color': 'white'}),
                html.P(f"Genres: {movie_details.get('Genre')}", style={'color': 'white'}),
                html.P(f"Cast: {movie_details.get('Actors')}", style={'color': 'white'})
            ]
        else:
            modal_content = [html.P("Movie details not found.", style={'color': 'white'})]

        return {'display': 'block'}, modal_content

    return {'display': 'none'}, []

if __name__ == '__main__':
    app.run_server(debug=True,jupyter_mode= 'external')


# Movie Page

In [ ]:
import os
import re
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import requests
from dash.dependencies import Input, Output
from sklearn.metrics.pairwise import cosine_similarity

# Function to fetch movie details (poster and cast)
def fetch_movie_details(imdbId, api_key):
    url = f"http://www.omdbapi.com/?i={imdbId}&apikey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get('Poster'), data.get('Actors')
    else:
        return None, None

# Your OMDb API key
api_key = '439d9f47'

# Load the datasets
ratings_df = pd.read_csv('/content/drive/MyDrive/rs_project/ml-latest-small/ratings.csv')
movies_df = pd.read_csv('/content/drive/MyDrive/rs_project/ml-latest-small/movies.csv')
links_df = pd.read_csv('/content/drive/MyDrive/rs_project/ml-latest-small/links.csv')
tags_df = pd.read_csv('/content/drive/MyDrive/rs_project/ml-latest-small/tags.csv')

# Preprocess movies data
def split_title_and_year(title):
    match = re.search(r'\((\d{4})\)', title)
    if match:
        year = match.group(1)
        name = title[:match.start()].strip()
        return name, year
    return title, None

movies_df[['title', 'year']] = movies_df['title'].apply(lambda x: pd.Series(split_title_and_year(x)))
movies_df['genres'] = movies_df['genres'].str.replace('|', ', ')
pre_movies_df = movies_df.copy()

# Convert IMDb IDs to formatted strings
links_df['imdbId'] = links_df['imdbId'].astype(str)
def format_imdb_id(imdb_id):
    if len(imdb_id) == 3:
        return 'tt0000' + imdb_id
    elif len(imdb_id) == 4:
        return 'tt000' + imdb_id
    elif len(imdb_id) == 5:
        return 'tt00' + imdb_id
    else:
        return 'tt0' + imdb_id

links_df['imdbId'] = links_df['imdbId'].apply(format_imdb_id)

# Merge movies and links data to include IMDb IDs
movies_df = pd.merge(movies_df, links_df[['movieId', 'imdbId']], on='movieId')

# Merge movies and ratings data
merged_df = pd.merge(movies_df, ratings_df, on='movieId')

# Create utility matrix
utility_matrix = merged_df.pivot(index='userId', columns='movieId', values='rating')
utility_matrix.fillna(0, inplace=True)

# Create similarity matrix
def get_similarity_matrix(utility_matrix):
    similarity_matrix = cosine_similarity(utility_matrix.T)
    similarity_matrix = pd.DataFrame(similarity_matrix, index=utility_matrix.columns, columns=utility_matrix.columns)
    return similarity_matrix

similarity_matrix = get_similarity_matrix(utility_matrix)

# Recommendation function
def recommend_movies(movie_title, top_n=10):
    movie_id = movies_df[movies_df['title'] == movie_title]['movieId'].values[0]
    similarity_scores = similarity_matrix[movie_id].sort_values(ascending=False)
    top_n_movies = similarity_scores.iloc[1:top_n+1]
    movie_titles = dict(zip(movies_df['movieId'], movies_df['title']))
    movie_years = dict(zip(movies_df['movieId'], movies_df['year']))
    movie_imdb_ids = dict(zip(movies_df['movieId'], movies_df['imdbId']))
    top_n_movies_df = pd.DataFrame({
        'Title': [movie_titles[movie_id] for movie_id in top_n_movies.index],
        'Year': [movie_years[movie_id] for movie_id in top_n_movies.index],
        'IMDbID': [movie_imdb_ids[movie_id] for movie_id in top_n_movies.index],
        'Similarity': top_n_movies.values
    })
    top_n_movies_df['Poster'], top_n_movies_df['Cast'] = zip(*top_n_movies_df['IMDbID'].apply(lambda x: fetch_movie_details(x, api_key)))

    return top_n_movies_df

app = dash.Dash(_name_, assets_folder='/content/drive/MyDrive/rs_project/assets')

# Define the layout of the app
app.layout = html.Div([
    html.Div(id='background-container', style={
        'backgroundImage': f'url({app.get_asset_url("background.png")})',
        'backgroundSize': 'cover',
        'backgroundPosition': 'center',
        'backgroundRepeat': 'no-repeat',
        'display': 'flex',
        'height': '100vh',
        'width': '100%',
        'color': 'white',
        'overflow': 'hidden',
        'position': 'relative',
        'top': '0',
        'left': '0',
        'zIndex': '1'
    }, children=[
        html.Div(style={'position': 'absolute', 'top': '0', 'left': '0', 'right': '0', 'bottom': '0', 'backgroundColor': 'rgba(0, 0, 0, 0.5)', 'zIndex': '1'}),

        html.Div([
            html.H1("Hello, Movie!\nPlease Identify Yourself", style={'textAlign': 'center', 'marginBottom': '20px', 'zIndex': '2', 'marginTop': '10px', 'fontSize': '30px', 'fontStyle': 'italic', 'fontWeight': 'bold'}),

            html.Div([
                dcc.Dropdown(
                    id='movie-dropdown',
                    options=[{'label': row['title'], 'value': row['title']} for index, row in pre_movies_df.iterrows()],
                    placeholder="Select a movie",
                    style={'width': '60%', 'marginBottom': '20px', 'color': 'black', 'zIndex': '2', 'margin': 'auto', 'fontSize': '15px'}
                ),
                dcc.Dropdown(
                    id='number-dropdown',
                    options=[{'label': str(i), 'value': i} for i in range(1, 21)],
                    placeholder="Select the number of top similar movies",
                    style={'width': '60%', 'marginBottom': '20px', 'color': 'black', 'zIndex': '2', 'margin': 'auto', 'fontSize': '15px'}
                ),
            ], style={'textAlign': 'center'}),

            html.Div(id='movie-details-container', style={'display': 'none'}, children=[
                html.Div([
                    html.Div(id='movie-data', style={'zIndex': '2'})
                ], style={'width': '100%', 'background': 'rgba(0, 0, 0, 0.6)', 'padding': '20px', 'borderRadius': '10px', 'zIndex': '0', 'overflowY': 'hidden'}),

                html.Div([
                    html.H2("Meet your similar movies", style={'textAlign': 'center', 'marginBottom': '20px', 'zIndex': '2', 'fontSize': '30px', 'fontStyle': 'italic', 'fontWeight': 'bold'}),
                    html.Div(id='similar-movies', style={'zIndex': '2'})
                ], style={'width': '100%', 'background': 'rgba(0, 0, 0, 0.6)', 'padding': '20px', 'borderRadius': '10px', 'zIndex': '0', 'overflowY': 'hidden','position': 'absolute'})
            ])
        ], style={'position': 'absolute', 'zIndex': '2'})
    ])
])

@app.callback(
    [Output('background-container', 'style'),
     Output('movie-details-container', 'style'),
     Output('movie-data', 'children'),
     Output('similar-movies', 'children')],
    [Input('movie-dropdown', 'value'),
     Input('number-dropdown', 'value')]
)
def update_movie_data(selected_movie, number):
    default_background_style = {
        'backgroundImage': f'url({app.get_asset_url("background.png")})',
        'backgroundSize': 'cover',
        'backgroundPosition': 'center',
        'backgroundRepeat': 'no-repeat',
        'display': 'flex',
        'flexDirection': 'column',
        'alignItems': 'center',
        'justifyContent': 'center',
        'height': '100vh',
        'width': '100%',
        'color': 'white',
        'overflow': 'hidden',
        'position': 'fixed',
        'top': '0', 'left': '0', 'zIndex': '0'
    }

    if not selected_movie or not number:
        return default_background_style, {'display': 'none'}, "", ""

    # Filter the movies DataFrame to get data for the selected movie
    selected_movie_data = pre_movies_df[pre_movies_df['title'] == selected_movie].iloc[0]

    # Fetch the poster and cast for the selected movie
    selected_movie_imdb_id = movies_df[movies_df['title'] == selected_movie]['imdbId'].values[0]
    selected_movie_poster, selected_movie_cast = fetch_movie_details(selected_movie_imdb_id, api_key)

    # Update the background image
    background_style = {
        'backgroundImage': f'url({selected_movie_poster})',
        'backgroundRepeat': 'no-repeat',
        'display': 'flex',
        'height': '100vh',
        'width': '100%',
        'color': 'white',
        'overflow': 'auto',
        'position': 'absolute',
        'backgroundSize': 'cover',
        'backgroundPosition': 'center',
        'top': '0',
        'left': '0',
        'zIndex': '0'
    }

    # Construct HTML table to display movie details
    movie_details_table = html.Table([
        html.Tbody([
            html.Tr([html.Td("Title:"), html.Td(selected_movie_data['title'])]),
            html.Tr([html.Td("Year:"), html.Td(selected_movie_data['year'])]),
            html.Tr([html.Td("Genres:"), html.Td(selected_movie_data['genres'])]),
            html.Tr([html.Td("Cast:"), html.Td(selected_movie_cast)])
        ])
    ], style={'color': 'white', 'fontSize': '20px'})

    # Call the recommend_movies function
    top_similar_movies = recommend_movies(selected_movie, number)

    similar_movies_list = html.Div(
        children=[
            html.Div([
                html.Img(src=movie['Poster'], style={'width': '150px', 'height': '225px', 'borderRadius': '10px', 'margin': '10px'}),
                html.P(f"{movie['Title']} ({movie['Year']})", style={'textAlign': 'center', 'color': 'white', 'margin': '10px'})
            ], style={'display': 'inline-block', 'margin': '10px'})
            for _, movie in top_similar_movies.iterrows()
        ],
        style={'display': 'flex', 'flexWrap': 'wrap', 'justifyContent': 'center', 'padding': '0px', 'background': 'rgba(0, 0, 0, 0.6)', 'list-style-type': 'none', 'maxHeight': '500px', 'overflowY': 'auto'}
    )
    return background_style, {'display': 'block'}, html.Div([
        html.Div([
            html.Img(src=selected_movie_poster, style={'width': '200px', 'height': '300px', 'borderRadius': '10px', 'marginBottom': '20px'}),
            movie_details_table
        ], style={'display': 'flex', 'flexDirection': 'row', 'alignItems': 'center', 'justifyContent': 'center'})
    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'height': '100%'}), similar_movies_list


if _name_ == '_main_':
    app.run_server(debug=True, jupyter_mode='external')
